In [20]:
import pandas as pd
import ccxt
from algos.daddy.huobi.HuobiDMService import HuobiDM
import json
import redis

In [21]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [22]:
pairs_df = pd.read_csv('pairs.csv')

In [23]:
pairs_df = pairs_df[pairs_df['types'].isnull()]

In [24]:
exchange_list = set(pairs_df['exchange'].values)

In [25]:
exchanges = {}

In [26]:
for exchange in exchange_list:
    if exchange == 'ftx':
        exchanges[exchange] = ccxt.ftx({})
    elif exchange == 'okex':
        exchanges[exchange] = ccxt.okex({})
    elif exchange == 'bybit':
        exchanges[exchange] = ccxt.bybit({})
    elif exchange == 'binance_futures':
        exchanges[exchange] = ccxt.binance({})
    elif exchange == 'huobi_swap':
        exchanges[exchange] = HuobiDM("https://api.hbdm.com", "", "")
    else:
        print(exchange)

In [28]:
for idx, row in pairs_df.iterrows():
    if "MOVE" in row['ccxt_symbol']:
        pairs = json.load(open('algos/vol_trend/pairs.json'))
    else:
        pairs = [row['ccxt_symbol']]
        
    for pair in pairs:
        if row['exchange'] in ['ftx', 'okex', 'bybit']:
            exchange = exchanges[row['exchange']]    
            book = exchange.fetch_order_book(pair)
            bid = book['bids'][0][0]
            ask = book['asks'][0][0]
        elif row['exchange'] == 'binance_futures':
            exchange = exchanges[row['exchange']] 
            book = exchange.fapiPublicGetDepth({'symbol': 'BTCUSDT', 'limit': 5})
            bid = book['bids'][0][0]
            ask = book['asks'][0][0]
        elif row['exchange'] == 'huobi_swap':
            book = exchanges['huobi_swap'].send_get_request('/swap-ex/market/depth', {'contract_code': row['ccxt_symbol'], 'type': 'step0'})['tick']
            bid = book['bids'][0][0]
            ask = book['asks'][0][0]
            
        if 'daddy' in row['feed']:
            r.set('{}_best_bid'.format(row['exchange']), bid)
            r.set('{}_best_ask'.format(row['exchange']), ask)
        elif 'vol_trend' in row['feed']:
            r.set('FTX_{}_best_bid'.format(pair), bid)
            r.set('FTX_{}_best_ask'.format(pair), ask)
        elif 'altcoin' in row['feed']:
            r.set('{}_best_bid'.format(pair), bid)
            r.set('{}_best_ask'.format(pair), ask)